In [1]:
import pandas as pd
from transformers import pipeline
from googletrans import Translator
from tqdm.notebook import tqdm
import time
import random
import logging
import numpy as np
from multiprocessing import Pool, cpu_count
import re

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
tqdm.pandas()

In [3]:
df_01 = pd.read_csv('/Volumes/KODAK/folder 02/language_translation/Language_translator/data/filtered_dataset.csv')

df_01.head()

,Comment
0,මොනා උනත් පොත්ත සුදු කෑල්ලක් දැකලා දෙකට නැවුනෙ...
1,බැරිනං PickMe එකෙන් අයින් වෙලා නිකං හයර් දුවපං...
2,ට් ‍ රිප් එක මැදදී කෑශ් හයර් එක කාඩ් හයර් එකට ...
3,me දුවපු කොල්ලෙක්ට විතරයි seen එක තේරෙන්නෙඋදේම...
4,මාත් හයර් දුවන්නෙ කස්ටමර්ගෙ පැත්තෙන් බලනකොට සා...


In [14]:
def clean_and_preprocess_text(text):
    """
    Comprehensive text cleaning and preprocessing for Sinhala-English mixed content
    Handles URLs, special chars, mixed language tokens, and whitespace
    """
    
    if not isinstance(text, str):
        return ""
    
    
    # Reular expression patterns
    sinhala_pattern = re.compile(r'[\u0D80-\u0DFF\u0DE6-\u0DEF0-9]+')
    english_pattern = re.compile(r'[a-zA-Z]+')
    url_pattern = re.compile(r'http\S+|www\S+|https\S+', flags=re.MULTILINE)
    special_char_pattern = re.compile(r'[[^\u0D80-\u0DFF\s.,!?\'\"]]')
    special_whitespace = re.compile(r'[\u200B-\u200D\uFEFF\u00A0]')
    
    # Remove URLs
    text = url_pattern.sub('', text)
    text = special_whitespace.sub(' ', text)
    
    # Split into tokens(words)
    tokens = text.split()
    processed_tokens = []
    
    for token in tokens:
        # Check langauge composition of each token
        has_sinhala = bool(sinhala_pattern.search(token))
        has_english = bool(english_pattern.search(token))
        
        # Pure Sinhala - process special chars only
        if has_sinhala and not has_english:
            cleaned_token = special_char_pattern.sub('', token)
            if cleaned_token:
                processed_tokens.append(cleaned_token)
                
        
        # Mixed Sinhala-English - extract only Sinhala part
        elif has_sinhala and has_english:
            sinhala_parts = ''.join(sinhala_pattern.findall(token))
            if sinhala_parts:
                processed_tokens.append(sinhala_parts)
                
                
        # Pure English - remove completely (change if you want to keep)
        else:
            cleaned_eng = special_char_pattern.sub('', token)
            if cleaned_eng:
                processed_tokens.append(f'[ENG: {cleaned_eng}]')
                
                
    # Reconstruct the text
    processed_text = ' '.join(processed_tokens)
    
    # Final cleaning and whitespace normalization 
    processed_text = special_char_pattern.sub('', processed_text)
    processed_text = ' '.join(processed_text.split())
    
    return processed_text.strip()

In [15]:
# load the pipeline

df = pd.read_csv('/Volumes/KODAK/folder 02/language_translation/Language_translator/data/filtered_dataset.csv')

# check the structure
print(df.head())
print(f'Total comments: {len(df)}')

# Apply the combined cleaning function
df['cleaned_comment'] = df['Comment'].apply(clean_and_preprocess_text)

# Remove empty comments after cleaning
df = df[df['cleaned_comment'].str.len() > 0]
print(f'Comments after cleaning: {len(df)}')


# Show some sample
for idx, row in df.head(5).iterrows():
    print(f'\nOriginal: {row['Comment']}')
    print(f'Cleaned: {row['cleaned_comment']}')

                                             Comment
0  මොනා උනත් පොත්ත සුදු කෑල්ලක් දැකලා දෙකට නැවුනෙ...
1  බැරිනං PickMe එකෙන් අයින් වෙලා නිකං හයර් දුවපං...
2  ට් ‍ රිප් එක මැදදී කෑශ් හයර් එක කාඩ් හයර් එකට ...
3  me දුවපු කොල්ලෙක්ට විතරයි seen එක තේරෙන්නෙඋදේම...
4  මාත් හයර් දුවන්නෙ කස්ටමර්ගෙ පැත්තෙන් බලනකොට සා...
Total comments: 807
Comments after cleaning: 807

Original: මොනා උනත් පොත්ත සුදු කෑල්ලක් දැකලා දෙකට නැවුනෙ නැති එකට respect කොල්ලො සමහර උන් නිකනුත් දාන් යන්න try කරනවනේ
Cleaned: මොනා උනත් පොත්ත සුදු කෑල්ලක් දැකලා දෙකට නැවුනෙ නැති එකට [ENG: respect] කොල්ලො සමහර උන් නිකනුත් දාන් යන්න [ENG: try] කරනවනේ

Original: බැරිනං PickMe එකෙන් අයින් වෙලා නිකං හයර් දුවපං මේකේ ඉන්න වියළි කලාපයේ උන් සල්ලි උස්සං හැම තැනම ගියාට කොළඹ ඉන්න එවුන් හැමතැනම ගෙනියන්නේ card කියලා නොදැන මෙතල වැළලෙන්ඩ හදනවා කෙල්ලගේ හමටයි රස්සාවටයි බැන බැන පිස්සු කෙලිනවා මෙයා කවුද කියලා මම දන්නෑ නමුත් මේ ප් ‍ රශ්නේ දැන් PickMe එකේ මල වදයක් වෙලා තියෙන්නේ එක්කෝ මගදි කියනවා trip එක cancel කරන්ඩ කියලා උන්ට companyයට ගෙවන්ඩ 

# Hybrid Approach

In [16]:
def translate_hybrid(text, max_retries = 3):
    if not text or len(text) == 0:
        return ""
    
    # Try Helsinki first
    for attempt in range(max_retries):
        try:
            result = translate_helsinki(text)
            if result and len(result) > 0:
                return result
        except:
            time.sleep(1)
            
            
    # Fallback to Google Translate
    for attempt in range(max_retries):
        try:
            result = translate_google(text)
            if result and len(result) > 0:
                return result
        except:
            time.sleep(1)
            
    return "" # Return empty string if all attempts fail

# Parallel Translation Function

In [17]:
def parallel_translate(df_chunk):
    return df_chunk['cleaned_comment'].progress_apply(translate_hybrid)

def translate_in_parallel(df, n_workers = None):
    if n_workers is None:
        n_workers = max(1, cpu_count() - 1)
        
        
    df_split = np.array_split(df, n_workers)
    
    with Pool(n_workers) as pool:
        results = pool.map(parallel_translate, df_split)
        
    # Combine results
    return pd.concat(results)